In [ ]:
!pip install -U tensorflow-addons
!pip3 install twint
!pip install whatthelang
!pip install TwitterAPI
!pip install Tweepy

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

import os
import tweepy
import requests

import pandas as pd
import numpy as np

from tweepy import OAuthHandler
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse
from requests_oauthlib import OAuth1

In [ ]:
DATASET_PATH = 'https://raw.githubusercontent.com/multimodal-entailment/acl-2021/main/dataset_placeholder.csv'

In [ ]:
df = pd.read_csv(DATASET_PATH)
df

In [ ]:
consumer_key = 'xxxx'
consumer_secret = 'yyyy'
access_token = '0000-aaaa'
access_secret = 'zzzz'
auth = OAuth1(consumer_key, consumer_secret,
              access_token, access_secret)

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [ ]:
def get_tweet_id(url):
  url_split = url.split('/status/')
  if len(url_split) < 2:
    print('Error getting id from url', url)
    return None
    
  return url_split[-1]

In [ ]:
def text_and_image_urls(api, tweet_id):
  try:
    tweet = api.get_status(tweet_id, tweet_mode="extended")
  except tweepy.TweepError as e:
    print('TweepError for', tweet_id, ':',
          e.args[0][0]['code'], e.args[0][0]['message'])
    return None

  try:  # Retweet.
    text = tweet.retweeted_status.full_text
    media_urls = []
    media = tweet.retweeted_status.entities.get('media', [])
    for m in media:
      media_urls.append(m['media_url'])
    return text, media_urls
  except AttributeError:  # Not a Retweet.
    text = tweet.full_text
    media_urls = []
    media = tweet.entities.get('media', [])
    for m in media:
      media_urls.append(m['media_url'])
    return text, media_urls

In [ ]:
tweets = []

for _, rows in df.iterrows():
  id_1, id_2 = get_tweet_id(rows['url_1']), get_tweet_id(rows['url_2'])
  if id_1 is None or id_2 is None:
    continue

  tweet1 = text_and_image_urls(api, id_1)
  if tweet1 is None:
    print(rows['url_1'])
    continue
  if len(tweet1[1]) != 1:
    print('Tweet', id1, 'without exactly one image')
    continue
  text_1, image_1 = tweet1[0], tweet1[1][0]

  tweet2 = text_and_image_urls(api, id_2)
  if tweet2 is None:
    print(rows['url_2'])
    continue
  if len(tweet2[1]) != 1:
    print('Tweet', id2, 'without exactly one image')
    continue
  text_2, image_2 = tweet2[0], tweet2[1][0]

  label = rows['label']
  
  tweets.append({
      'id_1': id_1,
      'text_1': text_1,
      'image_1': image_1,
      'id_2': id_2,
      'text_2': text_2,
      'image_2': image_2,
      'label' : label
  })

In [ ]:
contents_df = pd.DataFrame(tweets)
contents_df

In [ ]:
def download(url, path, filename):
  if not os.path.isdir(path):
    os.makedirs(path)
  response = requests.get(url, stream=True)
  file_size = int(response.headers.get("Content-Length", 0))
  extension = '.' + url.split('.')[-1]
  full_filename = os.path.join(path, filename + extension)
  progress = tqdm(response.iter_content(1024),
                  f"Downloading {full_filename}",
                  total=file_size,
                  unit="B",
                  unit_scale=True,
                  unit_divisor=1024)
  with open(full_filename, "wb") as f:
    for data in progress:
      f.write(data)
      progress.update(len(data))

In [ ]:
IMAGE_PATH = '/tmp/tweet_images/'
for _, row in contents_df.iterrows():
  download(row['image_1'], IMAGE_PATH, row['id_1'])
  download(row['image_2'], IMAGE_PATH, row['id_2'])

In [ ]:
image_file = os.path.join(IMAGE_PATH, contents_df['id_1'].iloc[0] + '.'
                          + contents_df['image_1'].iloc[0].split('.')[-1])
Image(image_file)